In [1]:
import BioSimSpace.Sandpit.Exscientia as BSS
import numpy as np
import pandas as pd

/home/yanfz/miniforge3/envs/a3fe/lib/python3.12/site-packages/nglview/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
INFO:rdkit:Enabling RDKit 2024.03.5 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.


In [2]:
system = BSS.IO.readMolecules(["bound/ensemble_equilibration_1/bound_preequil.prm7","bound/ensemble_equilibration_1/bound_preequil.rst7"])

In [3]:
system[0].getResidues()

[<BioSimSpace.Residue: name='LIG', molecule=52, index=0, nAtoms=127>]

In [4]:
lig = BSS.Align.decouple(system[0])
system.updateMolecule(0, lig)

In [5]:
traj = BSS.Trajectory.Trajectory(
    trajectory="bound/ensemble_equilibration_1/gromacs.xtc",
    topology="bound/ensemble_equilibration_1/bound_preequil.prm7",
)

Indexing XTC               7.0 s 197464768.0 bytes / s          


In [6]:
restraint = BSS.FreeEnergy.RestraintSearch.analyse(
    work_dir = "./get_restraints",
    system = system,
    traj = traj,
    temperature = 298 * BSS.Units.Temperature.kelvin,
    method="BSS",
    restraint_type = "multiple_distance",
)

INFO:MDAnalysis.core.universe:The attribute(s) types, masses have already been read from the topology file. The guesser will only guess empty values for this attribute, if any exists. To overwrite it by completely guessed values, you can pass the attribute to the force_guess parameter instead of the to_guess one
INFO:MDAnalysis.guesser.base:There is no empty types values. Guesser did not guess any new values for types attribute
INFO:MDAnalysis.guesser.base:There is no empty masses values. Guesser did not guess any new values for masses attribute


Searching for low variance pairs. Frame no:   0%|          | 0/2501 [00:00<?, ?it/s]

In [7]:
restrain_dict=restraint.toString(engine="Somd")

In [8]:
restrain_dict

'distance restraints dictionary = {(340, 22): (5.967301273865537, 20.0, 0.818216079516433), (6048, 5): (7.6177123280271895, 20.0, 0.8651516572845273), (4387, 57): (8.803827970449168, 20.0, 0.8765240203970492), (320, 10): (8.482756757352762, 20.0, 0.8829745202287498), (6109, 16): (5.147912057372962, 20.0, 0.8740407239436676), (6048, 33): (8.416228955107606, 20.0, 0.88667191628124), (3267, 65): (8.98063201127738, 20.0, 0.9363134171566152), (320, 24): (8.696532830731806, 20.0, 0.9607869088073446), (3267, 14): (8.46463697647875, 20.0, 0.932306182460616), (3243, 63): (9.279103367522826, 20.0, 0.9682310989346945), (4389, 0): (9.041243785614189, 20.0, 0.9672235054604617), (6154, 46): (6.0470674067987344, 20.0, 0.9859646528604333), (6154, 61): (7.2704204451744845, 20.0, 0.9656838947270607), (3267, 59): (7.0498180601990015, 20.0, 0.9917711232621951), (3279, 52): (6.557667833280758, 20.0, 0.9857389809777759), (4335, 53): (6.229698580999708, 20.0, 0.9629660760277075), (342, 20): (5.01244455439076

In [7]:
restraint.getCorrection(method = "numerical")

/home/yanfz/miniforge3/envs/a3fe/lib/python3.12/site-packages/BioSimSpace/Sandpit/Exscientia/FreeEnergy/_restraint.py:1157: UserWarning: The multiple distance restraint correction is assumes that only the 'permanent_distance_restraint' is active.
  _warnings.warn(


-0.5960 kcal/mol

In [10]:
view = BSS.Notebook.View(system)
view.molecules([0, 1, 2, 3])


NGLWidget(gui_style='ngl')

In [8]:
lam_vals_release_restraint = [0.000, 0.100, 0.200, 0.300, 0.400, 0.500, 0.600, 0.700, 0.800, 0.900, 1.000]


release_restraints_protocol = BSS.Protocol.FreeEnergy(runtime=1*BSS.Units.Time.nanosecond, 
                                                      timestep=4*BSS.Units.Time.femtosecond, 
                                                      lam_vals=lam_vals_release_restraint, 
                                                      perturbation_type="release_restraint")

release_restraints_fe_calc = BSS.FreeEnergy.AlchemicalFreeEnergy(restraint.system, 
                                                                 release_restraints_protocol, 
                                                                 engine='somd', 
                                                                 restraint=restraint,
                                                                 work_dir='./get_restraints/release1')
#generate one file, modify .cfg to be same as those of a3fe, then copy as five files.

/home/yanfz/miniforge3/envs/a3fe/lib/python3.12/site-packages/BioSimSpace/Sandpit/Exscientia/FreeEnergy/_alchemical_free_energy.py:370: UserWarning: SOMD simulations with some non-interacting ligands can be unstable. This affects some systems with multiple distance restraints during the release restraint state. If you experience problems, consider using multiple distance restraints with GROMACS instead.
  _warnings.warn(
